# IMDb

เว็บ IMDb รวบรวมข้อมูลเกี่ยวกับภาพยนตร์ต่างๆ และตัดข้อมูลบางส่วนมาให้ใช้ทำการทดลองต่างๆ ได้ https://www.imdb.com/interfaces/

โจทย์นี้คัดกรอง [ข้อมูลบางส่วนใน IMDb](https://drive.google.com/file/d/1V5VNg1WTMTS_eEOQqylektM9ZwdG1uq2/view?usp=sharing) มา โดยโดยมีไฟล์นี้อยู่

**Cast.tsv**
- tconst (string) - alphanumeric unique identifier of the title
- ordering (integer) – a number to uniquely identify rows for a given titleId
- nconst (string) - alphanumeric unique identifier of the name/person
- category (string) - the category of job that person was in
- job (string) - the specific job title if applicable, else '\N'
- characters (string) - the name of the character played if applicable, else '\N'

**Title.tsv**
- tconst (string) - alphanumeric unique identifier of the title
- titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
- primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
- originalTitle (string) - original title, in the original language
- isAdult (boolean) - 0: non-adult title; 1: adult title
- startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
- endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
- runtimeMinutes – primary runtime of the title, in minutes
- genres (string array) – includes up to three genres associated with the title

**Star.tsv**
- nconst (string) - alphanumeric unique identifier of the name/person
- primaryName (string)– name by which the person is most often credited
- birthYear – in YYYY format
- deathYear – in YYYY format if applicable, else '\N'
- primaryProfession (array of strings)– the top-3 professions of the person
- knownForTitles (array of tconsts) – titles the person is known for

## Preprocess

ใช้ encoding='utf-8' ตอนอ่านไฟล์

In [1]:
def read_table(fn:str):
    with open(fn,encoding='utf-8') as file:
        rows = list(map(lambda x: tuple(x.split('\t')),file.read().split('\n\n')))
        key = rows.pop(0)

    return [dict(zip(key,row))for row in rows]

In [2]:
df_star = read_table('Star.tsv')
df_cast = read_table('Cast.tsv')
df_title = read_table('Title.tsv')

# 1. มีภาพยนตร์กี่ประเภท (genres) แต่ละประเภทมีอย่างละกี่เรื่อง (เรียงตาม genre)
หาด้วย Title

```
Action 733
Adult 56
Adventure 277
Animation 132
Biography 17
8Comedy 1694
Crime 488
Documentary 484
Drama 3041
Family 247
Fantasy 218
History 120
Horror 559
Music 128
Musical 74
Mystery 268
News 7
Reality-TV 3
Romance 825
Sci-Fi 181
Sport 80
Thriller 726
War 83
Western 22
\N 270
```

In [3]:
genres_count = {}

In [4]:
for item in df_title:
    for genre in item['genres'].split(','):
        
        if genre not in genres_count:
            genres_count.setdefault(genre,0)
            
        genres_count[genre] += 1

In [5]:
for genre, count in sorted(genres_count.items(), key= lambda x: x[0]):
    print(f"{genre:<20}{count}")

Action              733
Adult               56
Adventure           277
Animation           132
Biography           178
Comedy              1694
Crime               488
Documentary         484
Drama               3041
Family              247
Fantasy             218
History             120
Horror              559
Music               128
Musical             74
Mystery             268
News                7
Reality-TV          3
Romance             825
Sci-Fi              181
Sport               80
Thriller            726
War                 83
Western             22
\N                  270


# 2. มีนักแสดงชายและหญิงอย่างละกี่คน
นับจากไฟล์ Star

```
actress 3537
actor 4300
```

In [6]:
profession_count = {}

In [7]:
for item in df_star:
    
    for prof in item['primaryProfession'].split(','):
        
        if 'act' not in prof:
            continue
        
        if prof not in profession_count:
            profession_count.setdefault(prof,0)
            
        profession_count[prof] += 1

In [8]:
for key, val in profession_count.items():
    print(f"{key:<20}{val}")

actress             3537
actor               4300


# 3. นักแสดงที่ยังมีชีวิตมีอายุเท่าไหร่บ้าง แต่ละช่วงมีกี่คน 
หาด้วย Star

```
32 213
33 233
34 288
35 328
36 317
37 365
38 389
39 438
40 430
41 448
42 495
43 460
44 442
45 463
46 472
47 447
48 505
49 511
50 498
```

In [9]:
age_count = {}

In [10]:
for item in df_star:
    
    if 'act' not in item['primaryProfession'] or item['deathYear'] != '\\N':
        continue
    
    age = 2021 - int(item['birthYear'])
        
    if age not in age_count:
        age_count.setdefault(age,0)
        
    age_count[age] += 1

In [11]:
for k, v in sorted(age_count.items(), key= lambda x: x[0]):
    print(k,v)

32 213
33 233
34 288
35 328
36 317
37 365
38 389
39 438
40 430
41 448
42 495
43 460
44 442
45 463
46 472
47 447
48 505
49 511
50 498


# 4. มีนักแสดงกี่คนที่เคยแสดงในภาพยนตร์ Action 
หาด้วย Cast + Title

```
992
```

In [23]:
action_titles = [item['tconst'] for item in df_title if 'Action' in item['genres']]

len(set(item["nconst"] for item in df_cast if (item['tconst'] in action_titles) and ('act' in item['category'])))

992

# 5. มีนักแสดงกี่คนที่แสดงในภาพยนตร์มากกว่า 1 ประเภท 
หาด้วย Cast + Title

```
3861
```

In [13]:
titles_morethan_1genre = [item['tconst'] for item in df_title if ',' in item['genres']]

len(set(item["nconst"] for item in df_cast if (item['tconst'] in titles_morethan_1genre) and ('act' in item['category'])))

3861

# 6. มีผู้กำกับกี่คนที่เป็นนักแสดงด้วย 
หาด้วย Star

```
1424
```

In [ ]:
count = 0
for item in df_star:
    if 'act' in item['primaryProfession'] and 'director' in 

In [14]:
len([item['nconst'] for item in df_star if 'act' in item['primaryProfession'] and 'director' in item['primaryProfession']])

1424

# 7. หาจำนวนภาพยนต์แนวรอมคอมในแต่ละปี (นับเฉพาะปีที่มีตั้งแต่ 1 เรื่องขึ้นไป)
หาด้วย Title

```
1991 3
1992 2
1993 3
1994 1
1995 4
1996 8
1997 3
1998 13
1999 8
2000 8
2001 7
2002 15
2003 9
2004 8
2005 15
2006 13
2007 15
2008 9
2009 10
2010 25
2011 24
2012 18
2013 14
2014 24
2015 21
2016 22
2017 10
2018 14
2019 15
2020 5
```

In [15]:
romcom_year_count = {}

In [16]:
for item in df_title:
    
    year = int(item['startYear'])
    
    if year not in romcom_year_count:
        romcom_year_count.setdefault(year, 0)
        
    if 'Comedy' in item['genres'] and 'Romance' in item['genres']:
        romcom_year_count[year] += 1

In [17]:
rc_items = tuple(romcom_year_count.items())
for y, c in rc_items:
    if not c >= 1:
        romcom_year_count.pop(y)

In [18]:
for k, v in sorted(romcom_year_count.items()):
    print(k,v)

1991 3
1992 2
1993 3
1994 1
1995 4
1996 8
1997 3
1998 13
1999 8
2000 8
2001 7
2002 15
2003 9
2004 8
2005 15
2006 13
2007 15
2008 9
2009 10
2010 25
2011 24
2012 18
2013 14
2014 24
2015 21
2016 22
2017 10
2018 14
2019 15
2020 5
